In [1]:
import pandas as pd
import requests
import collections

In [2]:
user_detail_data = pd.read_csv('users-details-2023.csv')
user_detail_df = pd.DataFrame(user_detail_data)
locations = user_detail_df['Location']

print(len(locations))
print(len(locations.unique()))
print(locations.nunique())

FileNotFoundError: [Errno 2] No such file or directory: 'users-details-2023.csv'

In [3]:
url = 'https://restcountries.com/v3.1/all'
response = requests.get(url)
countries = collections.defaultdict(str)

if response.status_code == 200:
    response_body = response.json()

    for i in range(len(response_body)):
        common = response_body[i]['name']['common']
        official = response_body[i]['name']['official']

        countries[common] = common

        if 'nativeNames' in response_body[i]:
            native_names = response_body[i]['name']['nativeName']

            for key, val in native_names.items():
                countries[val['common']] = common
                countries[val['official']] = common

        if 'translations' in response_body[i]:
            translations = response_body[i]['translations']

            for key, val in translations.items():
                countries[val['common']] = common
                countries[val['official']] = common
else:
    print("Fail")

us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
    'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
    'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
    'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
    'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

for state in us_states:
    countries[state] = 'United States'

In [ ]:
for index, row in user_detail_df.iterrows():

    location = row['Location']
    
    # To replace row whose type is not string with an empty string
    if type(location) != str:
        location = ""

    location_splited = location.split(",")
    is_valid_country = False

    for splited_str in location_splited:
        splited_str = splited_str.strip()

        if splited_str in countries:
            user_detail_df.at[index, 'Location'] = countries[splited_str]
            is_valid_country = True

    if not is_valid_country:
        user_detail_df.at[index, 'Location'] = None

print(locations.notna().sum())

In [ ]:
user_score_data = pd.read_csv('users-score-2023.csv')
user_score_df = pd.DataFrame(user_score_data)
common_users = pd.merge(user_score_df, user_detail_df, left_on=['user_id', 'Username'], right_on=['Mal ID', 'Username'], how='inner')

user_score_df = user_score_df[user_score_df.set_index(['user_id', 'Username']).index.isin(common_users.set_index(['user_id', 'Username']).index)]
user_detail_df = user_detail_df[user_detail_df.set_index(['Mal ID', 'Username']).index.isin(common_users.set_index(['user_id', 'Username']).index)]

print(user_score_df)
print(user_detail_df)
